In [ ]:
import numpy as np
import pandas as pd
import torch
import re
import torch
from tqdm import notebook
from itertools import combinations
from transformers import AutoTokenizer, AutoModel

from sklearn.metrics.pairwise import cosine_similarity

import json

import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

<ipython-input-2-1ed0aac1e065>:12: DeprecationWarning: Please use `pearsonr` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  from scipy.stats.stats import pearsonr


In [ ]:
posts = []
for line in open("ranking_test.jsonl", 'r'):
    posts.append(json.loads(line))
    
all_comments = []
for i in range(len(posts)):
    all_comments.append(pd.DataFrame.from_dict(posts[i], orient='columns'))
    
df = pd.concat(all_comments, ignore_index=True)


,text,comments
20739,JMAP – a better way to email,{'text': 'For the well acquainted -- how does ...
50918,Next-Generation Cloud: The Rise of the Unikernel,{'text': 'For those who are interested in this...
6105,Sacrificial Architecture,{'text': 'The sentence that I found the most i...
27636,A Slightly Skeptical View on Object-Oriented P...,{'text': 'That&#x27;s &quot;slightly skeptical...
56817,A400M Airbus Flier crashed because of software...,{'text': 'I am not surprised at all. There ar...
2011,Ask HN: Got an offer to leave my current emplo...,{'text': 'Caring about your direct reports and...
5133,Updated Secret Trans-Pacific Partnership Agree...,"{'text': 'As an IP lawyer, I&#x27;d like to sa..."
6496,Lecture 9: How to Raise Money,{'text': 'Notes: http:&#x2F;&#x2F;jonalmeida.c...
52351,What's wrong with finance,{'text': 'Lotsa issues.If financing (of some k...
736,Who's afraid of bromine?,{'text': 'This article is an industry puff pie...


In [ ]:
# оценка комментария (целевой признак)
df['score'] = df['comments'].map(lambda a: a['score'])
# Достать текст из словаря
df['comments'] = df['comments'].map(lambda a: a['text'])

In [ ]:
# удаление ссылок
df['comments'], df['text'] = df['comments'].str.lower(), df['text'].str.lower()
df['comments'] = df['comments'].apply(lambda x: re.sub(r'http\S+', '', x))
df['text'] = df['text'].apply(lambda x: re.sub(r'http\S+', '', x))

# удаление пунктуации
patterns = "[!#$%&()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
df['comments'] = df['comments'].apply(lambda x: re.sub(patterns, '', x))
df['text'] = df['text'].apply(lambda x: re.sub(patterns, '', x))

# удаление цифр
df['comments'] = df['comments'].str.replace('\d+', '')
df['text'] = df['text'].str.replace('\d+', '')

## Косинусная схожесть


In [ ]:
# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# переносим названия статей из колонки датафрейма в список и приводим все слова в нижний регистр
corpus = df['text'].str.lower().values

# удаляем из текста ненужные знаки препинания
corpus_new = []
for text in corpus:
    text = re.sub(r'\xa0', ' ', text)
    text = re.sub(r'«»,:', '', text)
    corpus_new.append(text)

# переводим предложения в векторы
# каждому слову соответсвует определеное число в словаре
tokenized = []
for i in corpus_new:
    tokenized.append((tokenizer.encode(i, add_special_tokens=True))[:512])

# рассчитываем максимальную длину маски внимания
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

# для каждого токенизированного предложения создаем маску внимания и приводим векторы к единой длине
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized])
attention_mask = torch.LongTensor(np.where(padded != 0, 1, 0))

# сколько предложений будет обработано за одну итерацию
batch_size = 1

# c помощью модели BERT переводим токенизированные предложения в эмбеддинги
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size * i : batch_size * (i + 1)]).cuda() 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size * i : batch_size * (i + 1)]).cuda()
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/70020 [00:00<?, ?it/s]

In [ ]:
corpus = df['comments'].str.lower().values

# удаляем из текста ненужные знаки препинания
corpus_new = []
for text in corpus:
    text = re.sub(r'\xa0', ' ', text)
    text = re.sub(r'«»,:', '', text)
    corpus_new.append(text)

# переводим предложения в векторы
# каждому слову соответсвует определеное число в словаре
tokenized = []
for i in corpus_new:
    tokenized.append((tokenizer.encode(i, add_special_tokens=True))[:512])

# рассчитываем максимальную длину маски внимания
max_len = 0
for i in tokenized:
    if len(i) > max_len:
        max_len = len(i)

# для каждого токенизированного предложения создаем маску внимания и приводим векторы к единой длине
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized])
attention_mask = np.where(padded != 0, 1, 0)

# сколько предложений будет обработано за одну итерацию
batch_size = 1

# c помощью модели BERT переводим токенизированные предложения в эмбеддинги
embeddings1 = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size * i : batch_size * (i + 1)]).cuda() 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size * i : batch_size * (i + 1)]).cuda()
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings1.append(batch_embeddings[0][:,0,:].cpu().numpy())

Token indices sequence length is longer than the specified maximum sequence length for this model (827 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/70020 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
f = open('cos_sim_test.txt', 'w')
for i in range(len(embeddings1)): 
    emb = str(cosine_similarity(embeddings[i], embeddings1[i])[0][0])
    f.write(emb + '\n')
f.close()

In [ ]:
from google.colab import files
files.download('cos_sim_test.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>